# Clonamos el repositorio para obtener los dataSet

In [2]:
!git clone https://github.com/joanby/tensorflow.git

Cloning into 'tensorflow'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 60311 (delta 32), reused 37 (delta 23), pack-reused 60260
Receiving objects: 100% (60311/60311), 442.46 MiB | 40.96 MiB/s, done.
Resolving deltas: 100% (82/82), done.
Checking out files: 100% (60225/60225), done.


# Damos acceso a nuestro Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Test it

In [ ]:
!ls '/content/drive/My Drive' 

# Google colab tools

In [3]:
from google.colab import files # Para manejar los archivos y, por ejemplo, exportar a su navegador
import glob # Para manejar los archivos y, por ejemplo, exportar a su navegador
from google.colab import drive # Montar tu Google drive

##Especificando la versión de TensorFlow

Ejecutando "importar tensorflow" importará la versión por defecto (actualmente 2.x). Puedes usar la 1.x ejecutando una celda con la "versión mágica de tensorflow" **antes de ejecutar "importar tensorflow".

### Si no funciona hacer el pip install


In [4]:
#!pip install tensorflow==1.14
%tensorflow_version 1.x

TensorFlow 1.x selected.


# Importar Tensorflow

In [5]:
import tensorflow as tf
print(tf.__version__)
import matplotlib.pyplot as plt

1.15.2


In [6]:
session = tf.Session()

# Redes neuronales multicapas

### Ejemplo 1D 

In [7]:
import numpy as np

In [8]:
data_size = 25
data_1d = np.random.normal(size = data_size)
x_input_1d = tf.placeholder(shape=[data_size], dtype=tf.float32)

In [9]:
def conv_layer_1d(input_1d, my_filter):
    input_2d = tf.expand_dims(input_1d, 0)
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    convolution = tf.nn.conv2d(input_4d, filter = my_filter, strides = [1,1,1,1], padding="VALID")
    output = tf.squeeze(convolution)
    return(output)

In [10]:
my_filter = tf.Variable(tf.random_normal(shape=[1,5,1,1]))
my_conv_output = conv_layer_1d(x_input_1d, my_filter)

In [11]:
def activation(input_1d):
    return tf.nn.relu(input_1d)
my_activation_output = activation(my_conv_output)

In [12]:
def max_pool(input_1d, width):
    input_2d = tf.expand_dims(input_1d, 0)
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    pooling = tf.nn.max_pool(input_4d, ksize=[1,1,width,1], strides=[1,1,1,1], padding="VALID")
    output = tf.squeeze(pooling)
    return(output)

In [13]:
my_maxpool_ouput = max_pool(my_activation_output, width=5)

In [14]:
def fully_connected(input_layer, num_output):
    weight_shape = tf.squeeze(tf.stack([tf.shape(input_layer), [num_output]]))
    weight = tf.random_normal(weight_shape, stddev=0.1)
    bias = tf.random_normal(shape=[num_output])
    input_layer_2d = tf.expand_dims(input_layer,0)
    full_output = tf.add(tf.matmul(input_layer_2d, weight), bias)
    full_output_1d = tf.squeeze(full_output)
    return(full_output_1d)

In [15]:
my_full_output = fully_connected(my_maxpool_ouput, 5)

In [16]:
init = tf.global_variables_initializer()
session.run(init)

In [17]:
feed_dict = {x_input_1d: data_1d}

In [18]:
data_1d

array([-1.07889055,  0.95052443,  1.33974865,  0.55979303, -0.08035685,
       -0.3821933 ,  0.24770011,  0.20264024,  0.3066495 , -0.49571026,
       -1.2334389 , -0.44657562, -0.44167007, -0.19813113,  0.57108106,
       -0.31243067, -0.11600661,  2.12846682, -0.33257253,  0.63222502,
       -1.30006191, -0.40697375, -0.60163739,  0.78884734,  1.27080713])

In [19]:
session.run(my_filter)

array([[[[-1.9801905e+00]],

        [[ 1.7667394e+00]],

        [[-1.9910543e+00]],

        [[-1.0889482e-03]],

        [[-1.3986280e+00]]]], dtype=float32)

In [20]:
# Operación de convolución
print("Input: tamaño 25, Operación: convolución con filtro de tamaño 5 + stride de tamaño 1, Resultado: tamaño 21")
print(session.run(my_conv_output, feed_dict=feed_dict))

Input: tamaño 25, Operación: convolución con filtro de tamaño 5 + stride de tamaño 1, Resultado: tamaño 21
[ 1.2600051  -0.09517754 -1.8499783  -0.7731869  -1.4384077   1.4839498
  0.9826251   1.7534258   1.5910469  -0.03081527  1.7343361   0.93483114
 -0.4499182  -0.9534517  -0.9890299  -4.70806     6.469946   -5.4905124
  5.2059374  -3.8411386   1.275001  ]


In [21]:
# Función de activación
print("Input: tamaño 21, Operación: ReLU al array anterior, Resultado: tamaño 21")
print(session.run(my_activation_output, feed_dict=feed_dict))

Input: tamaño 21, Operación: ReLU al array anterior, Resultado: tamaño 21
[1.2600051  0.         0.         0.         0.         1.4839498
 0.9826251  1.7534258  1.5910469  0.         1.7343361  0.93483114
 0.         0.         0.         0.         6.469946   0.
 5.2059374  0.         1.275001  ]


In [22]:
# Operción de Max Pooling
print("Input: tamaño 21, Operación: maxpooling con ventana de tamaño 5 + stride de tamaño 1, Resultado: tamaño 17")
print(session.run(my_maxpool_ouput, feed_dict=feed_dict))

Input: tamaño 21, Operación: maxpooling con ventana de tamaño 5 + stride de tamaño 1, Resultado: tamaño 17
[1.2600051  1.4839498  1.4839498  1.7534258  1.7534258  1.7534258
 1.7534258  1.7534258  1.7343361  1.7343361  1.7343361  0.93483114
 6.469946   6.469946   6.469946   6.469946   6.469946  ]


In [23]:
# Capa Totalmente Conectada
print("Input: tamaño 17, Operación de conectar totalmente la entrada con 5 valores de salida, Resultado: tamaño 5")
print(session.run(my_full_output, feed_dict=feed_dict))

Input: tamaño 17, Operación de conectar totalmente la entrada con 5 valores de salida, Resultado: tamaño 5
[ 3.3636603  -0.43062103  2.366209    2.5779343  -1.8963683 ]


### Ejemplo 2D

In [24]:
session = tf.Session()

In [25]:
data_size = [10,10]
data_2d = np.random.normal(size = data_size)
x_input_2d = tf.placeholder(dtype=tf.float32, shape=data_size)

In [26]:
def conv_layer_2d(input_2d, my_filter):
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    convolution = tf.nn.conv2d(input_4d, filter=my_filter, strides = [1,2,2,1], padding="VALID")
    output = tf.squeeze(convolution)
    return(output)

In [27]:
my_filter = tf.Variable(tf.random_normal(shape=[2,2,1,1]))
my_conv_output = conv_layer_2d(x_input_2d, my_filter)

In [28]:
def activation(input_2d):
    return(tf.nn.relu(input_2d))

In [29]:
my_activation_output = activation(my_conv_output)

In [30]:
def max_pool(input_2d, width, height):
    input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    pooling = tf.nn.max_pool(input_4d, ksize=[1,height,width,1], strides=[1,1,1,1], padding="VALID")
    output = tf.squeeze(pooling)
    return(output)

In [31]:
my_maxpool_ouput = max_pool(my_activation_output, width=2, height=2)

In [32]:
def fully_connected(input_layer, num_output):
    flat_input = tf.reshape(input_layer, [-1])
    weight_shape = tf.squeeze(tf.stack([tf.shape(flat_input), [num_output]]))
    weight = tf.random_normal(weight_shape, stddev=0.1)
    bias = tf.random_normal(shape=[num_output])
    input_layer_2d = tf.expand_dims(flat_input,0)
    full_output = tf.add(tf.matmul(input_layer_2d, weight), bias)
    full_output_1d = tf.squeeze(full_output)
    return(full_output_1d)

In [33]:
my_full_output = fully_connected(my_maxpool_ouput, 5)

In [34]:
init = tf.global_variables_initializer()
session.run(init)

In [35]:
feed_dict = {x_input_2d: data_2d}

In [36]:
data_2d

array([[ 0.64279357,  1.38886546, -0.73842588,  1.03282902, -1.35471834,
         0.70040449,  0.69265522, -0.28281848, -0.41040716,  0.09278582],
       [-1.81405803,  1.74568603, -0.26799812, -0.92388041,  0.21456775,
         0.64209968,  0.08102591,  1.87495642, -0.7742249 , -0.89105987],
       [-0.34429916,  0.53278324,  1.17835321, -1.00015473, -0.88002885,
        -1.39805838,  0.04031183, -0.95746534, -0.98438932, -0.84933427],
       [ 0.78833887,  0.04685028,  1.20546098, -1.39401358, -0.71973139,
         0.09947023, -0.2157645 ,  0.76169064,  0.7208307 ,  0.88837445],
       [-1.16058809,  0.11750307,  0.95880178,  1.10241473,  0.12027354,
         0.68099416,  0.34480902, -1.06520876, -0.64841115,  1.37800319],
       [-0.31953413,  1.41523442, -0.04855357, -0.32627329,  0.29845956,
         0.3902052 , -0.84021566,  0.05279293, -0.91419925,  0.76643293],
       [ 0.7613948 , -0.87322879,  1.12031335, -1.06113496, -1.2128738 ,
         1.56863508,  1.05188067, -0.83147871

In [37]:
session.run(my_filter)

array([[[[ 0.52747995]],

        [[-2.2563877 ]]],


       [[[ 1.6792697 ]],

        [[ 1.1640741 ]]]], dtype=float32)

In [38]:
# Operación de convolución
print("Input: tamaño 10x10, Operación: convolución con filtro de tamaño 2x2 + stride de tamaño 2x2, Resultado: tamaño 5x5")
print(session.run(my_conv_output, feed_dict=feed_dict))

Input: tamaño 10x10, Operación: convolución con filtro de tamaño 2x2 + stride de tamaño 2x2, Resultado: tamaño 5x5
[[-3.8089433  -4.245474   -1.1872021   3.3221624  -2.7632344 ]
 [-0.00540575  3.2798538   1.597532    2.7060142   3.6417847 ]
 [ 0.23353423 -2.443067   -0.5177232   1.23591    -4.094336  ]
 [ 0.1460381   1.4663992  -2.901121    0.50777775 -1.6751758 ]
 [ 1.1095675  -1.2370292  -0.9293361  -0.44480014 -2.3961778 ]]


In [39]:
# Función de activación
print("Input: tamaño 5x5, Operación: ReLU al array anterior, Resultado: tamaño 5x5")
print(session.run(my_activation_output, feed_dict=feed_dict))

Input: tamaño 5x5, Operación: ReLU al array anterior, Resultado: tamaño 5x5
[[0.         0.         0.         3.3221624  0.        ]
 [0.         3.2798538  1.597532   2.7060142  3.6417847 ]
 [0.23353423 0.         0.         1.23591    0.        ]
 [0.1460381  1.4663992  0.         0.50777775 0.        ]
 [1.1095675  0.         0.         0.         0.        ]]


In [40]:
# Operción de Max Pooling
print("Input: tamaño 5x5, Operación: maxpooling con ventana de tamaño 2x2 + stride de tamaño 1, Resultado: tamaño 4x4")
print(session.run(my_maxpool_ouput, feed_dict=feed_dict))

Input: tamaño 5x5, Operación: maxpooling con ventana de tamaño 2x2 + stride de tamaño 1, Resultado: tamaño 4x4
[[3.2798538  3.2798538  3.3221624  3.6417847 ]
 [3.2798538  3.2798538  2.7060142  3.6417847 ]
 [1.4663992  1.4663992  1.23591    1.23591   ]
 [1.4663992  1.4663992  0.50777775 0.50777775]]


In [41]:
# Capa Totalmente Conectada
print("Input: tamaño 4x4, Operación de conectar totalmente la entrada con 5 valores de salida, Resultado: tamaño 5")
print(session.run(my_full_output, feed_dict=feed_dict))

Input: tamaño 4x4, Operación de conectar totalmente la entrada con 5 valores de salida, Resultado: tamaño 5
[-0.36537176 -0.10842323  0.85886157 -0.22109967  0.07901031]
